# Ranking simulations based on quality evaluation based on order parameters

This notebook:

1. Ranks the individual simulations based on quality of order parameters of different fragments (headgroup, acyl chains and all order parameters)
2. Ranks simulations based on quality of invidual lipids

These rankings help to find the best simulations or force fields to analyze certain types of membranes or certain lipids.

# Initialize NMRlipids databank

In [83]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
#import mdtraj
import urllib.request
import yaml
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import dataframe_image as dfi

NMRlipidsDataBankFolder = '/media/osollila/Data/NMRlipids/Databank/'

sys.path.insert(1, NMRlipidsDataBankFolder + 'Scripts/BuildDatabank/')
from databankLibrary import download_link, lipids_dict, databank

path = NMRlipidsDataBankFolder + 'Data/Simulations/'
db_data = databank(path)
systems = db_data.get_systems()

lipid17 = ['Lipid14 with Verlet scheme']
gromosCKP = ['GROMOS-CKP, Berger/Chiu NH3 charges and PME', 'GROMOS-CKP, GROMOS NH3 charges and PME']
ECClipids = ['ECC-lipids, SPC water model, ECC-ions']
lipid17ff99 = ['Lipid17 and ff99 ions']
lipid17dang = ['Lipid17 and Dang ions']
BergerHoljte = ['Berger and Modified Höltje model for cholesterol']

rounding = ['headgroup', 'sn-1', 'sn-2', 'total']

def ShowTable(SortedQualities, quality):
    QualityTable = []
    for i in SortedQualities:
        StoredToTable = []
        #for dict_value in i[quality]:
        for k, v in i[quality].items():
            #print(k,v)
            if k in rounding:
                i[quality][k] = round(float(v))#, 2)
        #del i[quality]['Temperature']
        #del i[quality]['Number of molecules']
        molecules = ''
        MolNumbers = ''
        for lipid in i['system']['COMPOSITION']:
            #print(np.sum(i['system']['COMPOSITION'][lipid]['COUNT']))
            molecules = molecules + lipid + ':'
            MolNumbers = MolNumbers + str(np.sum(i['system']['COMPOSITION'][lipid]['COUNT']))  + ':'
        i[quality]['Molecules'] = molecules[:-1] + ' (' + MolNumbers[:-1] + ')'
        StoredToTable = i[quality]
        if i['system']['FF'] in lipid17:
            StoredToTable['Forcefield'] = 'Lipid17'
        elif i['system']['FF'] in gromosCKP:
            StoredToTable['Forcefield'] = 'GROMOS-CKP' 
        elif i['system']['FF'] in ECClipids:
            StoredToTable['Forcefield'] = 'ECClipids' 
        elif i['system']['FF'] in lipid17ff99:
            StoredToTable['Forcefield'] = 'Lipid17 (ff99 ions)' 
        elif i['system']['FF'] in lipid17dang:
            StoredToTable['Forcefield'] = 'Lipid17 (Dang ions)' 
        else:
            StoredToTable['Forcefield'] = i['system']['FF']
        
        #StoredToTable['Molecules'] = molecules[:-1]
        
        #if 'CHOL' in StoredToTable['Molecules']:
        #    continue
        #if '10.5281/zenodo.3741793' in StoredToTable['DOI']:
        #    continue
        #if '10.5281/zenodo.1306800' in StoredToTable['DOI']:
        #    continue
            
        AddData = True
        for content in QualityTable:
            if StoredToTable['Forcefield'] == content['Forcefield']:
                #print(StoredToTable['Forcefield'],content['Forcefield'])
                AddData = False
        
                
        #StoredToTable['Number of molecules'] = ' (' + MolNumbers[:-1] + ')'
        #StoredToTable['# molecules'] = ' (' + MolNumbers[:-1] + ')'
        #StoredToTable['Temperature'] = i['system']['TEMPERATURE']
        #StoredToTable['DOI'] = i['system']['DOI']
        if AddData:
            QualityTable.append(StoredToTable)    
    display(pd.json_normalize(QualityTable))

In [14]:
file = open(NMRlipidsDataBankFolder + 'Data/Ranking/POPC_sn-1_Ranking.json')
ranking = json.load(file)
#print(ranking[0])
#systems
ShowTable(ranking,'POPC')


file = open(NMRlipidsDataBankFolder + 'Data/Ranking/POPS_sn-1_Ranking.json')
ranking = json.load(file)
#print(ranking[0])
#systems
ShowTable(ranking,'POPS')

#for system in ranking:
#print(system[''])

,headgroup,sn-1,sn-2,total,Forcefield,Molecules,Number of molecules,Temperature,DOI
0,0,1,1,1,Berger,POPC:SOL (256:10342),(256:10342),300.00,10.5281/zenodo.1402417
1,1,1,1,1,Slipids,POPC:SOL (512:23943),(512:23943),298.00,10.5281/zenodo.166034
2,0,1,0,0,Berger and Modified Höltje model for cholesterol,POPC:CHOL:SOL (120:8:7290),(120:8:7290),298.00,10.5281/zenodo.4643899
3,1,1,0,1,ECC-lipids,POPC:SOL (128:6400),(128:6400),300.00,10.5281/zenodo.3335503
4,1,1,1,1,CHARMM36,POPC:CHOL:SOL (1024:128:57600),(1024:128:57600),298.15,10.5281/zenodo.5767451
5,1,1,1,1,MacRog,POPC:SOL (128:5120),(128:5120),300.00,10.5281/zenodo.3741793
6,1,1,0,1,slipids,CHOL:POPC:SOL (256:256:20334),(256:256:20334),298.00,10.5281/zenodo.159434
7,1,0,0,0,Lipid17,POPC:SOL (128:5120),(128:5120),298.15,10.5281/zenodo.30898
8,0,0,0,0,GROMOS-CKP,POPC:SOL (500:25000),(500:25000),298.00,10.5281/zenodo.5997385


,headgroup,sn-1,sn-2,total,Forcefield,Molecules,Number of molecules,Temperature,DOI
0,0,1,1,0,ECClipids,POPS:SOL:SOD (72:3600:72),(72:3600:72),298.0,10.5281/zenodo.1488094
1,0,1,0,0,GROMOS-CKP,POPS:SOL:SOD (128:4480:128),(128:4480:128),298.0,10.5281/zenodo.1129431
2,0,0,0,0,Slipids,POPS:SOL:SOD (128:4480:128),(128:4480:128),298.0,10.5281/zenodo.1129441
3,0,0,0,0,MacRog,POPS:SOL:SOD (128:4480:128),(128:4480:128),298.0,10.5281/zenodo.1283335
4,0,0,0,0,CHARMM36,POPS:SOL:SOD (128:4480:128),(128:4480:128),298.0,10.5281/zenodo.1129415
5,0,0,0,0,Lipid17 (ff99 ions),POPS:SOL:SOD (72:3600:72),(72:3600:72),298.0,10.5281/zenodo.1487906
6,0,0,0,0,Berger,POPS:SOL:SOD (128:4480:128),(128:4480:128),298.0,10.5281/zenodo.1129425
7,0,0,0,0,Lipid17 (Dang ions),POPS:SOL:SOD (72:3600:72),(72:3600:72),298.0,10.5281/zenodo.1487761


# Print best 13 simulations based on total order parameter ranking

In [104]:
with open(NMRlipidsDataBankFolder + 'Data/Ranking/SYSTEM_total_Ranking.json') as file:
    ranking = json.load(file)


TotalRanking = []
index = 0
for i in ranking:
    dictToBeAdded = {}
    dictToBeAdded['OPquality'] = i['TotalQuality']['total']
    dictToBeAdded['Headgroup'] = i['TotalQuality']['headgroup']
    dictToBeAdded['sn-1'] = i['TotalQuality']['sn-1']
    dictToBeAdded['sn-2'] = i['TotalQuality']['sn-2']
    try:
        dictToBeAdded['FFquality'] = i['TotalQuality']['FFQuality']
    except:
        dictToBeAdded['FFquality'] = ''
        
    if i['system']['FF'] in lipid17:
        dictToBeAdded['Force field'] = 'lipid17'
    elif i['system']['FF'] in gromosCKP:
        dictToBeAdded['Force field'] = 'GROMOS-CKP' 
    elif i['system']['FF'] in ECClipids:
         dictToBeAdded['Force field'] = 'ECClipids' 
    elif i['system']['FF'] in lipid17ff99:
            dictToBeAdded['Force field'] = 'Lipid17 (ff99 ions)' 
    elif i['system']['FF'] in lipid17dang:
        dictToBeAdded['Force field'] = 'Lipid17 (Dang ions)' 
    elif i['system']['FF'] in BergerHoljte:
        dictToBeAdded['Force field'] = 'Berger/Höltje' 
    else:
        dictToBeAdded['Force field'] = i['TotalQuality']['Forcefield']
        
    dictToBeAdded['Molecules'] = i['TotalQuality']['Molecules'] + i['TotalQuality']['Number of molecules']
    dictToBeAdded['Temperature'] = i['TotalQuality']['Temperature']
    dictToBeAdded['DOI'] = i['TotalQuality']['DOI']
    if index < 13:
        TotalRanking.append(dictToBeAdded)
    index += 1
    
TR = pd.json_normalize(TotalRanking)
TR.index = TR.index + 1
display(TR)
dfi.export(TR,"TotalRanking.png")

,OPquality,Headgroup,sn-1,sn-2,FFquality,Force field,Molecules,Temperature,DOI
1,0.76,0.61,0.84,0.82,514.766945,Slipids,POPC:SOL (512:23943),298.00,10.5281/zenodo.166034
2,0.69,0.63,0.70,0.76,713.024817,MacRog,POPC:SOL (128:5120),300.00,10.5281/zenodo.3741793
3,0.61,0.64,0.48,0.72,674.021485,MacRog,POPC:SOL (288:14400),298.00,10.5281/zenodo.13498
4,0.60,0.65,0.74,0.42,576.169287,ECC-lipids,POPC:SOL (128:6400),300.00,10.5281/zenodo.3335503
5,0.58,0.01,0.88,0.84,,Berger,POPC:SOL (256:10342),300.00,10.5281/zenodo.1402417
6,0.55,0.02,0.83,0.78,,Berger,POPC:SOL (128:7290),298.00,10.5281/zenodo.4643875
7,0.54,0.65,0.54,0.43,613.125521,ECC-lipids,POPC:SOL (128:6400),300.00,10.5281/zenodo.1118980
8,0.52,0.01,0.79,0.77,122.168365,Slipids,POPE:SOL (500:25000),310.00,10.5281/zenodo.3231342
9,0.51,0.10,0.87,0.57,140.087165,Slipids,POPE:SOL (336:13460),310.00,10.5281/zenodo.1293813
10,0.51,0.10,0.86,0.57,136.680061,Slipids,POPE:SOL (336:13460),310.00,10.5281/zenodo.1293813


[0406/134722.553792:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.
[0406/134722.558377:WARNING:bluez_dbus_manager.cc(248)] Floss manager not present, cannot set Floss enable/disable.
[0406/134722.703075:INFO:headless_shell.cc(659)] Written to file /tmp/tmp8dtkyrgg/temp.png.


In [105]:
with open(NMRlipidsDataBankFolder + 'Data/Ranking/POPE_total_Ranking.json') as file:
    ranking = json.load(file)


TotalRanking = []
index = 0
for i in ranking:
    dictToBeAdded = {}
    dictToBeAdded['OPquality'] = i['TotalQuality']['total']
    dictToBeAdded['Headgroup'] = i['TotalQuality']['headgroup']
    dictToBeAdded['sn-1'] = i['TotalQuality']['sn-1']
    dictToBeAdded['sn-2'] = i['TotalQuality']['sn-2']
    try:
        dictToBeAdded['FFquality'] = i['TotalQuality']['FFQuality']
    except:
        dictToBeAdded['FFquality'] = ''
        
    if i['system']['FF'] in lipid17:
        dictToBeAdded['Force field'] = 'lipid17'
    elif i['system']['FF'] in gromosCKP:
        dictToBeAdded['Force field'] = 'GROMOS-CKP' 
    elif i['system']['FF'] in ECClipids:
         dictToBeAdded['Force field'] = 'ECClipids' 
    elif i['system']['FF'] in lipid17ff99:
            dictToBeAdded['Force field'] = 'Lipid17 (ff99 ions)' 
    elif i['system']['FF'] in lipid17dang:
        dictToBeAdded['Force field'] = 'Lipid17 (Dang ions)' 
    elif i['system']['FF'] in BergerHoljte:
        dictToBeAdded['Force field'] = 'Berger/Höltje' 
    else:
        dictToBeAdded['Force field'] = i['TotalQuality']['Forcefield']
        
    dictToBeAdded['Molecules'] = i['TotalQuality']['Molecules'] + i['TotalQuality']['Number of molecules']
    dictToBeAdded['Temperature'] = i['TotalQuality']['Temperature']
    dictToBeAdded['DOI'] = i['TotalQuality']['DOI']
    if index < 13:
        TotalRanking.append(dictToBeAdded)
    index += 1
    
TR = pd.json_normalize(TotalRanking)
TR.index = TR.index + 1
display(TR)
dfi.export(TR,"TotalRankingPOPE.png")

,OPquality,Headgroup,sn-1,sn-2,FFquality,Force field,Molecules,Temperature,DOI
1,0.52,0.01,0.79,0.77,122.168365,Slipids,POPE:SOL (500:25000),310.0,10.5281/zenodo.3231342
2,0.51,0.10,0.87,0.57,140.087165,Slipids,POPE:SOL (336:13460),310.0,10.5281/zenodo.1293813
3,0.51,0.10,0.86,0.57,136.680061,Slipids,POPE:SOL (336:13460),310.0,10.5281/zenodo.1293813
4,0.44,0.54,0.52,0.27,197.034217,CHARMM36,SOL:POPE (5760:144),310.0,10.5281/zenodo.2641987
5,0.25,0.53,0.14,0.07,234.339081,CHARMM36,SOL:POPE (25000:500),310.0,10.5281/zenodo.3237461
6,0.19,0.09,0.12,0.35,264.439235,Lipid17,POPE:SOL (500:25000),310.0,10.5281/zenodo.4424292


[0406/134839.634963:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.
[0406/134839.640037:WARNING:bluez_dbus_manager.cc(248)] Floss manager not present, cannot set Floss enable/disable.
[0406/134839.780719:INFO:headless_shell.cc(659)] Written to file /tmp/tmpbz_75h5f/temp.png.
